# Following [SpaGE Tutorial](https://github.com/tabdelaal/SpaGE/blob/master/SpaGE_Tutorial.ipynb)

Integration of *osmFISH* spatial data with the *AllenSSp* scRNA-seq data

In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt

import scipy
import scipy.stats as st
from scipy import linalg
from scipy import sparse as sp
import pickle
from sklearn.metrics.pairwise import euclidean_distances

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/Volumes/LaCie/school/combine_lab/SpaGE/SpaGE_Datasets/'

# import spatial data

In [3]:
ds = loompy.connect(path + 'Spatial/osmFISH/osmFISH_SScortex_mouse_all_cells.loom')
FISH_Genes = ds.ra['Gene']   
colAtr = ds.ca.keys()

df = pd.DataFrame()
for i in colAtr:
    df[i] = ds.ca[i]

osmFISH_meta = df.iloc[np.where(df.Valid == 1)[0], :]
osmFISH_data = ds[:,:]
osmFISH_data = osmFISH_data[:,np.where(df.Valid == 1)[0]]
osmFISH_data = pd.DataFrame(data= osmFISH_data, index= FISH_Genes)

del ds, colAtr, i, df, FISH_Genes

# Select cortical regions only to match the AllenSSp dataset
Cortex_Regions = ['Layer 2-3 lateral', 'Layer 2-3 medial', 'Layer 3-4', 
                  'Layer 4','Layer 5', 'Layer 6', 'Pia Layer 1']
Cortical = np.stack(i in Cortex_Regions for i in osmFISH_meta.Region)

osmFISH_meta = osmFISH_meta.iloc[Cortical,:]
osmFISH_data = osmFISH_data.iloc[:,Cortical]
del Cortex_Regions,Cortical

In [4]:
osmFISH_meta

,CellID,ClusterID,ClusterName,Region,Total_molecules,Valid,X,Y,_tSNE_1,_tSNE_2,size_pix,size_um2
0,778,18,Inhibitory CP,Layer 6,390,1,18171.230942,24590.795275,-58.132385,5.219181,60911.0,257.348975
2,3642,18,Inhibitory CP,Layer 4,405,1,11247.433715,36626.892415,-61.572853,-1.185364,44143.0,186.504175
5,769,18,Inhibitory CP,Layer 4,491,1,18374.295796,26286.856654,-66.862244,-1.766500,41014.0,173.284150
7,5870,18,Inhibitory CP,Layer 4,518,1,4197.789280,39465.355903,-59.923431,-3.739808,42376.0,179.038600
9,5682,18,Inhibitory CP,Layer 2-3 medial,210,1,3757.866349,46281.167166,-64.769470,5.166343,23922.0,101.070450
...,...,...,...,...,...,...,...,...,...,...,...,...
4832,3635,25,Vascular Smooth Muscle,Layer 3-4,59,1,11457.806019,38036.523162,-12.763592,7.068469,7993.0,33.770425
4833,3336,25,Vascular Smooth Muscle,Pia Layer 1,144,1,9214.047907,45314.585476,0.060634,30.242371,14865.0,62.804625
4834,6462,25,Vascular Smooth Muscle,Layer 6,236,1,4476.301656,31097.786260,3.578825,28.862700,24835.0,104.927875
4835,260,25,Vascular Smooth Muscle,Layer 2-3 lateral,67,1,18796.948664,36603.475262,-11.011883,5.821001,8873.0,37.488425


In [5]:
# Normalization of ST gene expression
cell_count = np.sum(osmFISH_data,axis=0)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

osmFISH_data_norm = osmFISH_data.apply(Log_Norm_spatial,axis=0)

# scale
genes = st.zscore(osmFISH_data_norm)

In [6]:
cell_st = osmFISH_meta.loc[:,['CellID', 'X', 'Y']]

# add gene expression info for each cell at spot i
cell_spot = cell_st.merge(genes.T, left_index=True, right_index=True, how='left')

cell_spt = cell_spot.reset_index(drop=True) # reset index
cell_spt

,CellID,X,Y,Gad2,Slc32a1,Crhbp,Cnr1,Vip,Cpne5,Pthlh,...,Ctps,Anln,Mrc1,Hexb,Ttr,Foxj1,Vtn,Flt1,Apln,Acta2
0,778,18171.230942,24590.795275,1.621371,1.777401,-1.285498,0.169445,-0.417264,0.932325,-1.285498,...,0.817961,0.050672,-0.237032,0.876704,-0.237032,-0.417264,-0.635124,-1.285498,-0.237032,-0.417264
1,3642,11247.433715,36626.892415,2.723553,2.088070,0.162081,-0.609545,-0.353881,-0.609545,-0.609545,...,0.162081,-0.150690,-0.609545,-0.150690,-0.609545,-0.609545,-0.353881,-0.954585,0.017943,-0.609545
2,769,18374.295796,26286.856654,2.459380,2.289477,1.116087,-0.466125,-0.148632,-0.951611,0.186618,...,-0.022988,-0.148632,-0.293912,1.150048,-0.677581,-0.148632,-0.022988,-0.951611,-0.466125,-0.293912
3,5870,4197.789280,39465.355903,2.253989,1.549556,-0.197524,0.225092,-0.510714,-0.982660,0.135831,...,0.225092,-0.982660,-0.072863,0.515976,-0.072863,-0.717495,-0.717495,-0.982660,-0.717495,-0.072863
4,5682,3757.866349,46281.167166,1.496840,2.316222,-0.944582,1.345124,0.013995,-0.944582,-0.944582,...,0.495107,-0.354909,0.013995,0.283120,-0.354909,-0.354909,-0.354909,-0.354909,0.283120,-0.944582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,3635,11457.806019,38036.523162,0.309345,-0.843207,-0.843207,-0.843207,0.309345,0.309345,-0.843207,...,1.309683,-0.843207,1.086314,-0.843207,-0.843207,-0.843207,-0.843207,-0.843207,0.309345,1.086314
3401,3336,9214.047907,45314.585476,1.396286,-0.977202,-0.977202,1.229384,1.229384,-0.977202,-0.977202,...,0.381686,0.759097,0.110551,-0.278649,-0.278649,-0.977202,1.024502,0.381686,-0.278649,2.598400
3402,6462,4476.301656,31097.786260,0.232061,-0.423666,-1.342403,-0.423666,-0.156431,-1.342403,-0.423666,...,0.055889,0.232061,2.117512,-0.784520,0.830838,0.232061,-0.423666,-1.342403,0.232061,1.848433
3403,260,18796.948664,36603.475262,0.632525,-1.041085,-1.041085,1.198118,0.632525,0.129437,-1.041085,...,0.129437,0.129437,-1.041085,-1.041085,0.129437,-1.041085,-1.041085,-1.041085,0.632525,1.389190


# Get ddf(x) for all training samples

## Create adj mat (inspired by [GCNG](https://github.com/xiaoyeye/GCNG/blob/master/data_generation_interaction_ten_fold.py) and [SpaGCN](https://github.com/jianhuupenn/SpaGCN))


From [GCNG](https://github.com/xiaoyeye/GCNG/blob/2fba06bac64e97744a9a8d2a1505aa5c2117730c/data_generation_interaction_ten_fold.py#L19)

In [ ]:
#### from GCNG ####

############ the distribution of distance
distance_list = []
print ('calculating distance matrix, it takes a while')

# compute euclidean distance, considering both physical location and gene expression
for j in range(cell_st.shape[0]):
    for i in range(cell_st.shape[0]):
        if i!=j:
            distance_list.append(np.linalg.norm(cell_spot.iloc[j, 1:]-cell_spot.iloc[i, 1:]))

# save out file
np.save(path+'dist_array_eucl_xyz.npy',np.array(distance_list))

# np.savetxt("dist_list.csv", 
#            distance_list,
#            delimiter ="\t")

In [ ]:
distance_array = np.array(distance_list)
distance_array.shape

import saved distance array (instead of re-running since it takes a long time to compute)

In [ ]:
distance_array = np.load(path+'dist_array_eucl_xyz.npy')
print(distance_array.shape)
distance_array

from [spektral](https://github.com/danielegrattarola/spektral/blob/1dbdf9e919ff4606ee5375567a9b993687c95bcd/spektral/utils/convolution.py#L47) -- unable to import the package (received 'illegal hardware' error)

In [7]:
### from spektral ###
def degree_power(A, k):
    r"""
    Computes \(\D^{k}\) from the given adjacency matrix. Useful for computing
    normalised Laplacian.
    :param A: rank 2 array or sparse matrix.
    :param k: exponent to which elevate the degree matrix.
    :return: if A is a dense array, a dense array; if A is sparse, a sparse
    matrix in DIA format.
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        degrees = np.power(np.array(A.sum(1)), k).ravel()
    degrees[np.isinf(degrees)] = 0.0
    if sp.issparse(A):
        D = sp.diags(degrees)
    else:
        D = np.diag(degrees)
    return D

def normalized_adjacency(A, symmetric=True):
    r"""
    Normalizes the given adjacency matrix using the degree matrix as either
    \(\D^{-1}\A\) (row normalization) or \(\D^{-1/2}\A\D^{-1/2}\) (symmetric normalization).
    :param A: rank 2 array or sparse matrix;
    :param symmetric: boolean, compute symmetric normalization;
    :return: the normalized adjacency matrix.
    
    motivation for symmetric norm: takes into account both # of neighbours and their own # of neighbours
    """
    if symmetric:
        normalized_D = degree_power(A, -0.5)
        return normalized_D.dot(A).dot(normalized_D)
    else:
        normalized_D = degree_power(A, -1.0) # row normalized: consider only nieghbour connected to node v
        return normalized_D.dot(A)

In [8]:
distance_matrix = euclidean_distances(cell_spot.iloc[:, 1:], cell_spot.iloc[:, 1:])

############### get normalized sparse adjacent matrix
dist_symnorm = normalized_adjacency(distance_matrix, symmetric=True)

In [9]:
print(distance_matrix)

[[    0.         13885.48235918  1708.18015734 ... 15162.19227364
  12028.96694124 19405.05509302]
 [13885.48235918     0.         12558.20499189 ...  8741.81396431
   7549.55471867 10852.35417449]
 [ 1708.18015734 12558.20499189     0.         ... 14707.1188864
  10325.27560572 18883.65330954]
 ...
 [15162.19227364  8741.81396431 14707.1188864  ...     0.
  15342.54223372  4269.96297519]
 [12028.96694124  7549.55471867 10325.27560572 ... 15342.54223372
      0.         18196.03282577]
 [19405.05509302 10852.35417449 18883.65330954 ...  4269.96297519
  18196.03282577     0.        ]]


In [10]:
print(dist_symnorm)

[[0.00000000e+00 3.85134799e-04 3.76706298e-05 ... 3.66614173e-04
  2.89538098e-04 4.37072874e-04]
 [3.85134799e-04 0.00000000e+00 3.61531199e-04 ... 2.75929298e-04
  2.37218049e-04 3.19089077e-04]
 [3.76706298e-05 3.61531199e-04 0.00000000e+00 ... 3.69097647e-04
  2.57955898e-04 4.41460057e-04]
 ...
 [3.66614173e-04 2.75929298e-04 3.69097647e-04 ... 0.00000000e+00
  4.20261086e-04 1.09447929e-04]
 [2.89538098e-04 2.37218049e-04 2.57955898e-04 ... 4.20261086e-04
  0.00000000e+00 4.64291023e-04]
 [4.37072874e-04 3.19089077e-04 4.41460057e-04 ... 1.09447929e-04
  4.64291023e-04 0.00000000e+00]]


In [11]:
unweighted_Adj = np.zeros(distance_matrix.shape) # initialize A w/ zeros

for i in range(unweighted_Adj.shape[0]):
    for j in range(unweighted_Adj.shape[1]):
        if dist_symnorm[i,j] > 0:
            unweighted_Adj[i,j] = 1
unweighted_Adj

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [12]:
A_weighted = np.zeros(distance_matrix.shape) # initialize A with zeros
l = 0.5

for i in range(A_weighted.shape[0]):
    for j in range(A_weighted.shape[1]):
        if i!=j:
            A_weighted[i,j] = unweighted_Adj[i,j] / (( (distance_matrix[i,j]**2) / (2*l**2) ) ) 
A_weighted

array([[0.00000000e+00, 2.59327195e-09, 1.71357319e-07, ...,
        2.17493361e-09, 3.45551945e-09, 1.32782317e-09],
       [2.59327195e-09, 0.00000000e+00, 3.17040591e-09, ...,
        6.54284880e-09, 8.77257985e-09, 4.24543393e-09],
       [1.71357319e-07, 3.17040591e-09, 0.00000000e+00, ...,
        2.31161124e-09, 4.68993365e-09, 1.40216127e-09],
       ...,
       [2.17493361e-09, 6.54284880e-09, 2.31161124e-09, ...,
        0.00000000e+00, 2.12410191e-09, 2.74234308e-08],
       [3.45551945e-09, 8.77257985e-09, 4.68993365e-09, ...,
        2.12410191e-09, 0.00000000e+00, 1.51013781e-09],
       [1.32782317e-09, 4.24543393e-09, 1.40216127e-09, ...,
        2.74234308e-08, 1.51013781e-09, 0.00000000e+00]])

In [13]:
# distance_matrix_threshold_I_N = np.float32(whole_distance_matrix_threshold_I) ## do not normalize adjcent matrix
distance_matrix_threshold_I_N = np.float32(A_weighted)
distance_matrix_threshold_I_N_crs = sp.csr_matrix(distance_matrix_threshold_I_N)
with open(path+'symmnorm_weighted_adj_mat_crs.pickle', 'wb') as fp:
    pickle.dump(distance_matrix_threshold_I_N_crs, fp)

In [14]:
distance_matrix_threshold_I_N

array([[0.0000000e+00, 2.5932720e-09, 1.7135731e-07, ..., 2.1749336e-09,
        3.4555194e-09, 1.3278232e-09],
       [2.5932720e-09, 0.0000000e+00, 3.1704059e-09, ..., 6.5428489e-09,
        8.7725800e-09, 4.2454338e-09],
       [1.7135731e-07, 3.1704059e-09, 0.0000000e+00, ..., 2.3116113e-09,
        4.6899338e-09, 1.4021613e-09],
       ...,
       [2.1749336e-09, 6.5428489e-09, 2.3116113e-09, ..., 0.0000000e+00,
        2.1241020e-09, 2.7423431e-08],
       [3.4555194e-09, 8.7725800e-09, 4.6899338e-09, ..., 2.1241020e-09,
        0.0000000e+00, 1.5101378e-09],
       [1.3278232e-09, 4.2454338e-09, 1.4021613e-09, ..., 2.7423431e-08,
        1.5101378e-09, 0.0000000e+00]], dtype=float32)

In [40]:
# explore some distance metrics
print(max(map(max, distance_matrix_threshold_I_N)))
print(len(distance_matrix_threshold_I_N[np.where(distance_matrix_threshold_I_N > 0.000001)])/2)
print(min(distance_matrix_threshold_I_N[np.where(distance_matrix_threshold_I_N != 0)]))
print(len(distance_matrix_threshold_I_N[np.where(distance_matrix_threshold_I_N < 0.000001)])/2)

1337.4694
24522.0
3.4643227e-10
5772174.5
